<img src="ml2_group_assignment.png" width="800"/>

# <font color=green> Introduction </font>

This is a continuation of forest_cover_type_detector_gr_a_Part1.

Above we import the files created in the previous notebook so that this notebook can run independently

Table of contents is an extension of the previous notebook

# <font color=green> Table of contents </font>

* [Libaries used](#0)
* [0. Import Data](#0)
* [__5. Feature Engineering__](#5) 
  * [5.1 Check for Anomalies and Outliers](#5.1)
       * [5.1.1 Outlier Detection Treatment using Inter-Quartile Range rule Function](#5.1.1)
       * [5.1.2 Inter-Quartile Range rule: 4 IQR from Median](#5.1.2)
       * [5.1.3 Inter-Quartile Range rule: 3 IQR from Median](#5.1.3)
  * [5.2 Feature Transformation and Building of new features](#5.2)
      * [5.2.1 Bivariate Combinations](#5.2.1)  
      * [5.2.2 Polynominal](#5.2.2)
      * [5.2.3 ID](#5.2.3)  
      * [5.2.4 Distance to Hydrology](#5.2.4)   
      * [5.2.5 Horizontal Distance To Roadways ](#5.2.5) 
      * [5.2.6 Slope](#5.2.6)  
      * [5.2.7 Horizontal Distance To Fire Points ](#5.2.7)  
      * [5.2.8 Hillshade](#5.2.8) 
          * [5.2.8.1 Mean Hillshade](#5.2.8.1) 
          * [5.2.8.2 Hillshade 9am](#5.2.8.2)
          * [5.2.8.3 Hillshade Noon](#5.2.8.3)          
          * [5.2.8.4 Hillshade 3pm](#5.2.8.4)       
          * [5.2.8.5 Hillshade Ratios](#5.2.8.5)        
      * [5.2.9 Geoclimate Groping](#5.2.9) 
* [__6. Feature Selection__](#6)
  * [6.0 Prepare Data and Standardization](#6.1)
  * [6.1 Single tree](#6.2) 
  * [6.2 Bagging](#6.2) 
  * [6.3 Random Forest](#6.3)  
  * [6.4 Extra Trees](#6.4)
       * [6.4.1 Feature Number Selecion](#6.6.1)       
       * [6.4.2 Lasso Regularization](#6.6.2)
       * [6.4.3 Filter Methods](#6.6.3)   
  * [6.5 Recursive Feature Elimination](#6.5)  
  * [6.6 Tree Based Methodologies](#6.6) 
       * [6.6.1 RandomForestClassifier](#6.6.1)       
       * [6.6.2 XGBoost](#6.6.2)
       * [6.6.3 Extra trees Classifier](#6.6.3)       
  * [6.7 Score of all methods Together](#6.7) 


<a id='0'></a>
# <font color=green> Libraries used </font>

In [ ]:
#!pip install squarify
#!pip install htmltabletomd
#!pip install GraphViz
#!pip install pygraphviz

In [ ]:
import pandas as pd
import numpy as np
import math 
import seaborn as sns  # Graphing
import matplotlib.pyplot as plt
import squarify #treemap
import matplotlib.pyplot as plt
import warnings
import plotly.graph_objects as go
import xgboost as xgb
import scipy.stats as stats
import htmltabletomd
import pydotplus

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import KBinsDiscretizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestRegressor


from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression

from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

from sklearn import datasets
from sklearn import linear_model

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import f_classif

from sklearn.preprocessing import MinMaxScaler

from scipy.stats import norm


from yellowbrick.target import FeatureCorrelation
from yellowbrick.classifier import ROCAUC
from yellowbrick.model_selection import rfecv


from plotly.subplots import make_subplots
from IPython.display import Image  
from io import StringIO


from itertools import combinations

from dtreeviz.trees import *


from numpy import percentile

warnings.simplefilter(action='ignore', category=FutureWarning)

<a id='0'></a>
##  <font color=green>0.Import the Data </font>

Let’s load the original Kaggle training and test data and create a data frame

In [ ]:
data_test = pd.read_csv("train.csv")
data_test = pd.read_csv("test.csv")

Let's keep the original dataset for later comparisons and make a copy for the FE process

In [ ]:
df_original = data_test.copy()

<a id='5'></a>
<a id='5.1'></a>
# <font color=green>  5.Feature Engineering<font>

<a id='5.2'></a>
## <font color=green> 5.2 Feature Transformation and Building of new features <font>

<a id='5.2.1'></a>
### <font color=green> 5.2.1 Bivariate Combinations <font>

During feature engineering, we want to try to create a wide variety of interactions between multiple variables in order to create new variables. 


By manipulating them together, we create opportunities to have new and impactful features which could potentially impact our target variable, thus engineering our features. 

For this argument, we will create as many bivariate combinations of our predicting variables using the ‘combinations’ method from itertools library.

We will not make interactions with the dummy variables as these are either 0 or 1 and we will not get any additional information from making the interaction this way. 

Furthermore, it is not recommended to use standardization before bivariate combinations as we want to increase the signal. <br>

Sources: https://towardsdatascience.com/feature-engineering-combination-polynomial-features-3caa4c77a755 <br>

https://samchaaa.medium.com/preprocessing-why-you-should-generate-polynomial-features-first-before-standardizing-892b4326a91d

In order to use the bivariate combination we split the dataset for using it.Note this is not the split we will use later for testing the algorithm. This has only the purpose of testing all the combination and selecting the best once. 

In [ ]:
# Identify and drop our target variable 'Cover_Type' from dataframe
X = data_test

Train Test Split (80/20 size), drop duplicates and missing values


Here, we create every possible bivariate combination to be tested for feature engineering, no dummies


We take categorical variables prior to our feature engineering

In [ ]:
column_list = X.columns
filtered_column_list = [column for column in column_list if 'Soil_Type' not in column and 'Wilderness_Area' not in column and 'Id' not in column ] 
interactions = list(combinations(filtered_column_list, 2))
interactions

Addition and division has been taken out as it created a lot of noise in the data. The division makes sense if it has a business meaning and the addition only if it is the same scale. 

However we will add the variables which have the same metrics together in a second step but not in a for loop. 

In [ ]:
for (key, value) in interactions:
    data_test[key + '_x_' + value] = data_test[key] * data_test[value]
    #data_test[key + '_+_' + value] = data_test[key] + data_test[value]
    #data_test[key + '_divide_' + value] = data_test[key] / data_test[value]

In [ ]:
pd.set_option('display.max_columns', None) #to make all columns visible in dataframe now that we have many
data_test.head(1)

<a id='5.2.2'></a>
### <font color=green> 5.2.2 Polynomial Features <font>
    
We have just seen how to make two variables interact together, but sometimes the relationship between dependent and independent variables are more complex and not linear. 
    
Polynomials is another way to create new features! A very strong option for new features is increasing the power of a single variable. 
    
For our purposes, we will try and see if all the existing variables, can improve our Baseline by being increased to the  power.<br>
Source: https://towardsdatascience.com/feature-engineering-combination-polynomial-features-3caa4c77a755

Here we select only the columns we are interested in, this is from column 2 to 9

In [ ]:
X_pf = data_test.iloc[:, 1:10]
X_pf.head(1)

By default, when running polynomials we lose the name of our labels. This function gives us the ability to preserve it with the name + the transformation done

In [ ]:
def PolynomialFeatures_labeled(input_df,power):
    '''Basically this is a cover for the sklearn preprocessing function. 
    The problem with that function is if you give it a labeled dataframe, it ouputs an unlabeled dataframe with potentially
    a whole bunch of unlabeled columns. 

    Inputs:
    input_df = Your labeled pandas dataframe (list of x's not raised to any power) 
    power = what order polynomial you want variables up to. (use the same power as you want entered into pp.PolynomialFeatures(power) directly)

    Ouput:
    Output: This function relies on the powers_ matrix which is one of the preprocessing function's outputs to create logical labels and 
    outputs a labeled pandas dataframe   
    '''
    poly = PolynomialFeatures(power)
    output_nparray = poly.fit_transform(input_df)
    powers_nparray = poly.powers_

    input_feature_names = list(input_df.columns)
    target_feature_names = ["Constant Term"]
    for feature_distillation in powers_nparray[1:]:
        intermediary_label = ""
        final_label = ""
        for i in range(len(input_feature_names)):
            if feature_distillation[i] == 0:
                continue
            else:
                variable = input_feature_names[i]
                power = feature_distillation[i]
                intermediary_label = "%s^%d" % (variable,power)
                if final_label == "":         #If the final label isn't yet specified
                    final_label = intermediary_label
                else:
                    final_label = final_label + " x " + intermediary_label
        target_feature_names.append(final_label)
    output_df = pd.DataFrame(output_nparray, columns = target_feature_names)
    return output_df



Polynominal features of degree two gives us 46 new features. Since we have already enough information, we will not go for Polynominal three to avoid dimensionality issues later on

In [ ]:
test_df_pw2 = PolynomialFeatures_labeled(X_pf,2)
pd.set_option('display.max_columns', None)
test_df_pw2.shape

There are some fields duplicated with respect to the original df. This is a side effect of the function as normal features are replicated to the power of one which is still the same value so we delete these

In [ ]:
column_list = test_df_pw2.columns
cols = [column for column in column_list if '^1' not in column]
test_df_pw2=test_df_pw2[cols]
test_df_pw2.head(1)

In [ ]:
X

In here, we concatenate our output to consolidate the ponlynomials with the feature combinations

In [ ]:
data_test = pd.concat([data_test,test_df_pw2], axis=1)
data_test.head(1)

This results in a huge dataset. Just for curiosity we ran polynomials to the power of three and see that just the resulting dataframe is already bigger than the consolidated one above

<a id='5.2.3'></a>
### <font color=green> 5.2.3 ID <font>

We agree for the test to not remove ID because the ID is the unique indentifier to evaluate


For the train we will remove it as it doesn't add any value to the model

In [ ]:
data_test.drop('Id',axis = 1, inplace = True)

<a id='5.2.4'></a>
### <font color=green> 5.2.4 Distance To Hydrology <font>
#### <font color=green> New Features <font>

We combine Vertical distance to Hydrology and Horizontal distance to Hydrology since these two are highly correlated. This suggests to attempt a diagonal distance to hidrology using the Pythagoras theorem.

We will call this newly engineered feature, Distance_To_Hydrology
 
Source : https://towardsdatascience.com/types-of-transformations-for-better-normal-distribution-61c22668d3b9

In [ ]:
data_test['Distance_To_Hydrology'] = data_test['Horizontal_Distance_To_Hydrology']**2 +data_test['Vertical_Distance_To_Hydrology']**2
data_test['Distance_To_Hydrology'] = data_test['Distance_To_Hydrology']**0.5
data_test.head()

#### <font color=green> Square root and logarithm Transformation  <font>

Now we are checking the distribution of the newly created variable and see if further transformation is needed. 

The Distance to Hydrology inherits skewness from parent variables. It is positively skewed and has zero values. 

In order to use log we will use log + 1 in order to use logarithm with zero values. 

Source: https://www.youtube.com/watch?v=_c3dVTRIK9c and 

Source_2: https://towardsdatascience.com/types-of-transformations-for-better-normal-distribution-61c22668d3b9

As a rule of thumb, the skewness can be interpreted as follows:
<img src="Skew.png" width=400 height=200 align="center">

Source: https://www.marsja.se/transform-skewed-data-using-square-root-log-box-cox-methods-in-python/

We do some transformations to minimize skewness

In [ ]:
#Using the log10+ 1 logarithm 
data_test['log10_Distance_To_Hydrology'] = np.log10(data_test['Distance_To_Hydrology']+1)

In [ ]:
#Using the square root 
data_test['sqr_Distance_To_Hydrology'] = data_test['Distance_To_Hydrology']**0.5

#### <font color=green> Results after logarithm and <font color=darkcyan> Square root Transformation<font>

As you can see above, for distance to Hydrology the __square root__ showed a better performance in terms of skewness and is closer to a normal bell shaped than the logarithm transformation. We will be using Square Root as a new feature in the dataset and will frop the others from the dataset.  

For now we only remove log10 but later we will delete also originals

In [ ]:
data_test.drop(['log10_Distance_To_Hydrology'], axis=1,inplace=True)

<a id='5.2.5'></a>
### <font color=green> 5.2.5 Horizontal Distance To Roadways <font>

#### <font color=green> Square root and logarithm Transformation  <font>

For log transformation there should be no zeros, negative values and the distribution should be positive skewed( bigger than 1 is positive) hence we are using the square root as you can see for logarithm transformation below the distribution did not improve!!!

#### <font color=green> Results after logarithm and <font color=darkcyan> Square root Transformation<font>

In [ ]:
# since we have null values we add plus 1 to avoid log of zero.We are using natural log and log10
data_test['Sqr_Horizontal_Distance_To_Roadways'] = data_test['Horizontal_Distance_To_Roadways']**0.5
data_test['log_Horizontal_Distance_To_Roadways'] = np.log(data_test['Horizontal_Distance_To_Roadways']+1)
data_test['log10_Horizontal_Distance_To_Roadways'] = np.log10(data_test['Horizontal_Distance_To_Roadways']+1)

We achieved the best result again using square root of the Horizontal Distance to Roadways. Similarly as before, we remove failed experiments

In [ ]:
data_test.drop(['log_Horizontal_Distance_To_Roadways','log10_Horizontal_Distance_To_Roadways'], axis=1,inplace=True)

<a id='5.2.6'></a>
### <font color=green> 5.2.6 Slope <font>
#### <font color=green> Square root and logarithm Transformation  <font>

In [ ]:
print('\033[95m'+ "Skew before transformation\n", data_test['Slope'].skew(), 
      "\nmin\n", data_test['Slope'].min(),
      "\nmax \n", data_test['Slope'].max(),)

#### <font color=green> Results after logarithm and <font color=darkcyan> Sqrare root Transformation<font>

In [ ]:
# since we have null values we add plus 1 to avoid log of zero
data_test['logSlope'] = np.log(data_test['Slope']+1)

In [ ]:
data_test['SqrSlope'] = data_test['Slope']**0.5

Since the skweness for the slope shows better performance when using the square root, we will transform the variable into square root as well. 

In [ ]:
data_test.drop(['logSlope'], axis=1,inplace=True)

<a id='5.2.7'></a>
### <font color=green> 5.2.7 Horizontal Distance To Fire Points  <font>
#### <font color=green> Transformation  <font>

#### <font color=green> Results after logarithm and <font color=darkcyan> Sqrare root Transformation<font>

In [ ]:
# since we have null values we add plus 1 to avoid log of zero
data_test['log_Horizontal_Distance_To_firepoints'] = np.log(data_test['Horizontal_Distance_To_Fire_Points']+1)

In [ ]:
#Transform with square root
data_test['sqr_Horizontal_Distance_To_firepoints'] = data_test['Horizontal_Distance_To_Fire_Points']**0.5

Since square root transformation gives the best result in skewness, we will also use sqr for the feature variable.

In [ ]:
data_test.drop(['log_Horizontal_Distance_To_firepoints'], axis=1,inplace=True)

<a id='5.2.8'></a>
### <font color=green> 5.2.8 Hillshades <font>
<a id='5.2.8.1'></a>
### <font color=green> 5.2.8.1 Mean Hillshade <font>
#### <font color=green> Creation of new Feature: Mean Hillshade <font>

In [ ]:
# We take the average of Hillshades,which gives you the average light exposure of each cover type during the day
data_test['Mean_Hillshade'] = (data_test['Hillshade_9am']+data_test['Hillshade_Noon']+data_test['Hillshade_3pm'])/3

In [ ]:
#Itensity of the Hillshade variables in 3 bin siizes with the bin discretizer
est = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')
data_test['Mean_Hillshade_bin'] = est.fit_transform(data_test[['Mean_Hillshade']])

In [ ]:
data_test[['Mean_Hillshade_bin','Mean_Hillshade']].describe()

In [ ]:
print('\033[95m'+"Skew before transformation\n", data_test['Mean_Hillshade'].skew(), 
      "\nmin\n", data_test['Mean_Hillshade'].min(),
      "\nmax\n", data_test['Mean_Hillshade'].max(),)

#### <font color=green> Results after logarithm Transformation <font color=darkcyan>, Square root Transformation<font color=gold> and BoxCox Transformation<font>

In [ ]:
data_test['log_Mean_Hillshade'] = np.log(data_test['Mean_Hillshade'])

In [ ]:
print('\033[92m'+"Skew after log transformation\n", data_test['log_Mean_Hillshade'].skew(), 
      "\nmin\n", data_test['log_Mean_Hillshade'].min(),
      "\nmax\n", data_test['log_Mean_Hillshade'].max(),)

In [ ]:
data_test['log10Mean_Hillshade'] = np.log10(data_test['Mean_Hillshade'])

In [ ]:
print('\033[92m'+"Skew after log10 transformation\n", data_test['log10Mean_Hillshade'].skew(), 
      "\nmin\n", data_test['log10Mean_Hillshade'].min(),
      "\nmax\n", data_test['log10Mean_Hillshade'].max(),)

In [ ]:
data_test['sqr_Mean_Hillshade'] = data_test['Mean_Hillshade']**0.5

In [ ]:
print('\033[96m'+"Skew after Square Root transformation\n", data_test['sqr_Mean_Hillshade'].skew(), 
      "\nmin\n", data_test['sqr_Mean_Hillshade'].min(),
      "\nmax\n", data_test['sqr_Mean_Hillshade'].max(),)

In [ ]:
#Now, the Box-Cox transformation also requires our data to only contain positive numbers
# transform training data with Boxcox
data_test['Mean_Hillshade_boxcox'], _ = stats.boxcox(data_test['Mean_Hillshade'])

In [ ]:
print('\033[93m'+"Skew after Boxcox transformation\n", data_test['Mean_Hillshade_boxcox'].skew(), 
      "\nmin\n", data_test['Mean_Hillshade_boxcox'].min(),
      "\nmax\n", data_test['Mean_Hillshade_boxcox'].max(),)

In [ ]:
stats.normaltest(data_test['Mean_Hillshade_boxcox'])

In [ ]:
from scipy.stats import norm
import scipy.stats as stats

def histPlot(first_feature,col):
    sns.distplot(first_feature,color=col,fit = norm,kde = True,kde_kws = {'shade': True, 'linewidth': 3});

f = plt.figure(figsize=(20,10))
f.add_subplot(331)
histPlot(data_test['Mean_Hillshade'], 'purple')
f.add_subplot(335)
histPlot(data_test['log_Mean_Hillshade'], 'green')
f.add_subplot(334)
histPlot(data_test['Mean_Hillshade_boxcox'], 'gold')                    
f.add_subplot(332)
histPlot(data_test['sqr_Mean_Hillshade'], 'c')

The distribution did not improve with Square Root and Logarithms Transformation. Hence we use BoxCox which improved the distribution substantially. 

In [ ]:
data_test.drop(['log10Mean_Hillshade','log_Mean_Hillshade','sqr_Mean_Hillshade'], axis=1,inplace=True)

<a id='5.2.8.2'></a>
### <font color=green> 5.2.8.2 Hillshade 9am <font>
#### <font color=green> Transformation  <font>

In [ ]:
print('\033[95m'+"Skew before transformation\n", data_test['Hillshade_9am'].skew(), 
      "\nmin\n", data_test['Hillshade_9am'].min(),
      "\nmax\n", data_test['Hillshade_9am'].max(),)

#### <font color=green> Results after logarithm Transformation <font color=darkcyan>, Square root Transformation<font color=gold> and BoxCox Transformation<font>

In [ ]:
data_test['log_Hillshade_9am'] = np.log(data_test['Hillshade_9am']+1)

In [ ]:
print('\033[92m'+"Skew after log transformation\n", data_test['log_Hillshade_9am'].skew(), 
      "\nmin\n", data_test['log_Hillshade_9am'].min(),
      "\nmax\n", data_test['log_Hillshade_9am'].max(),)

In [ ]:
data_test['sqr_Hillshade_9am'] = data_test['Hillshade_9am']**0.5

In [ ]:
print('\033[96m'+"Skew after Square Root transformation\n", data_test['sqr_Hillshade_9am'].skew(), 
      "\nmin\n", data_test['sqr_Hillshade_9am'].min(),
      "\nmax\n", data_test['sqr_Hillshade_9am'].max(),)

In [ ]:
#Now, the Box-Cox transformation also requires our data to only contain positive numbers, transform training data with Boxcox
data_test['Hillshade_9am_boxcox'], lam  = stats.boxcox(data_test['Hillshade_9am']+1)
#lam is the best lambda for the distribution

In [ ]:
print('\033[93m'+"Skew after Boxcox transformation\n", data_test['Hillshade_9am_boxcox'].skew(), 
      "\nmin\n", data_test['Hillshade_9am_boxcox'].min(),
      "\nmax\n", data_test['Hillshade_9am_boxcox'].max(),)

In [ ]:
def histPlot(first_feature,col):
    sns.distplot(first_feature,color=col,fit = norm,kde = True,kde_kws = {'shade': True, 'linewidth': 3});

f = plt.figure(figsize=(20,10))
f.add_subplot(331)
histPlot(data_test['Hillshade_9am'], 'purple')
f.add_subplot(335)
histPlot(data_test['log_Hillshade_9am'], 'green')
f.add_subplot(334)
histPlot(data_test['Hillshade_9am_boxcox'], 'gold')                    
f.add_subplot(332)
histPlot(data_test['sqr_Hillshade_9am'], 'c')

BoxCox outperforms the other two for the Hillshade 9am 

In [ ]:
data_test.drop(['log_Hillshade_9am','sqr_Hillshade_9am'], axis=1,inplace=True)

<a id='5.2.8.3'></a>
### <font color=green> 5.2.8.3 Hillshade Noon <font>
#### <font color=green> Transformation  <font>

In [ ]:
print('\033[95m'+"Skew before transformation\n", data_test['Hillshade_Noon'].skew(), 
      "\nmin\n", data_test['Hillshade_Noon'].min(),
      "\nmax\n", data_test['Hillshade_Noon'].max(),)

#### <font color=green> Results after logarithm Transformation <font color=darkcyan>, Square root Transformation<font color=gold> and BoxCox Transformation<font>

In [ ]:
data_test['log_Hillshade_Noon'] = np.log(data_test['Hillshade_Noon']+1)

In [ ]:
data_test['sqr_Hillshade_Noon'] = data_test['Hillshade_Noon']**0.5

Box Coc is outperforming the other transformations for Hillshade Noon

In [ ]:
data_test.drop(['log_Hillshade_Noon','sqr_Hillshade_Noon'], axis=1,inplace=True)

<a id='5.2.8.4'></a>
### <font color=green> 5.2.8.4 Hillshade 3pm <font>
#### <font color=green> Transformation  <font>

In [ ]:
print('\033[95m'+"Skew before transformation\n", data_test['Hillshade_3pm'].skew(), 
      "\nmin\n", data_test['Hillshade_3pm'].min(),
      "\nmax\n", data_test['Hillshade_3pm'].max(),)

#### <font color=green> Results after logarithm Transformation <font color=darkcyan>, Square root Transformation<font color=gold> and BoxCox Transformation<font>

In [ ]:
data_test['log_Hillshade_3pm'] = np.log(data_test['Hillshade_3pm']+1)

In [ ]:
print('\033[92m'+"Skew after log transformation\n", data_test['log_Hillshade_3pm'].skew(), 
      "\nmin\n", data_test['log_Hillshade_3pm'].min(),
      "\nmax\n", data_test['log_Hillshade_3pm'].max(),)

In [ ]:
data_test['sqr_Hillshade_3pm'] = data_test['Hillshade_3pm']**0.5

In [ ]:
print('\033[96m'+"Skew after Square Root transformation\n", data_test['sqr_Hillshade_3pm'].skew(), 
      "\nmin\n", data_test['sqr_Hillshade_3pm'].min(),
      "\nmax\n", data_test['sqr_Hillshade_3pm'].max(),)

In [ ]:
#Now, the Box-Cox transformation also requires our data to only contain positive numbers, transform training data with Boxcox
data_test['Hillshade_3pm_boxcox'], lam  = stats.boxcox(data_test['Hillshade_3pm']+1)
#lam is the best lambda for the distribution

In [ ]:
print('\033[93m'+"Skew after Boxcox transformation\n", data_test['Hillshade_3pm_boxcox'].skew(), 
      "\nmin\n", data_test['Hillshade_3pm_boxcox'].min(),
      "\nmax\n", data_test['Hillshade_3pm_boxcox'].max(),)

In [ ]:
def histPlot(first_feature,col):
    sns.distplot(first_feature,color=col,fit = norm,kde = True,kde_kws = {'shade': True, 'linewidth': 3});

f = plt.figure(figsize=(20,10))
f.add_subplot(331)
histPlot(data_test['Hillshade_3pm'], 'purple')
f.add_subplot(335)
histPlot(data_test['log_Hillshade_3pm'], 'green')
f.add_subplot(334)
histPlot(data_test['Hillshade_3pm_boxcox'], 'gold')                    
f.add_subplot(332)
histPlot(data_test['sqr_Hillshade_3pm'], 'c')

For the Hillshade 3pm the data was not highly skwed, we either keep the original or we can use boxcox as it improved the variables as well. 

In [ ]:
data_test.drop(['log_Hillshade_3pm','sqr_Hillshade_3pm'], axis=1,inplace=True)

<a id='5.2.8.5'></a>
### <font color=green> 5.2.8.5 Hillshades  Ratios <font>

In [ ]:
data_test['ratio_Hillshade_3pm'] = data_test['Hillshade_3pm']/255
data_test['ratio_Hillshade_Noon'] = data_test['Hillshade_Noon']/255
data_test['ratio_Hillshade_9am'] = data_test['Hillshade_9am']/255

In [ ]:
data_test['Hillshade_9am_bin'] = data_test['Hillshade_9am'] > 150
data_test['Hillshade_Noon_bin'] = data_test['Hillshade_Noon'] > 150
data_test['Hillshade_3pm_bin'] = data_test['Hillshade_3pm'] > 150

In [ ]:
data_test['Hillshade_9am_bin'] = data_test['Hillshade_9am_bin'].astype(int)
data_test['Hillshade_Noon_bin'] = data_test['Hillshade_Noon_bin'].astype(int)
data_test['Hillshade_3pm_bin'] = data_test['Hillshade_3pm_bin'].astype(int)

In [ ]:
data_test['Hillshade_allday_bin'] = data_test['Hillshade_9am_bin'] * data_test['Hillshade_Noon_bin'] * data_test['Hillshade_3pm_bin']

In [ ]:
data_test['Hillshade_allday_bin'].value_counts()

<a id='5.2.8.6'></a>
### <font color=green> 5.2.8.6 Aspect <font>
#### <font color=green> Transformation  <font>

In [ ]:
print('\033[95m'+"Skew before transformation\n", data_test['Aspect'].skew(), 
      "\nmin\n", data_test['Aspect'].min(),
      "\nmax\n", data_test['Aspect'].max(),)

#### <font color=green> Results after logarithm Transformation <font color=darkcyan>and Square root Transformation<font>

In [ ]:
data_test['sqr_Aspect'] = data_test['Aspect']**0.5

In [ ]:
print('\033[96m'+"Skew after Square Root transformation\n", data_test['sqr_Aspect'].skew(), 
      "\nmin\n", data_test['sqr_Aspect'].min(),
      "\nmax\n", data_test['sqr_Aspect'].max(),)

In [ ]:
data_test['log_Aspect'] = np.log(data_test['Aspect']+1)

In [ ]:
print('\033[92m'+"Skew after log transformation\n", data_test['log_Aspect'].skew(), 
      "\nmin\n", data_test['log_Aspect'].min(),
      "\nmax\n", data_test['log_Aspect'].max(),)

In [ ]:
def histPlot(first_feature,col):
    sns.distplot(first_feature,color=col,fit = norm,kde = True,kde_kws = {'shade': True, 'linewidth': 3});

f = plt.figure(figsize=(20,10))
f.add_subplot(331)
histPlot(data_test['Aspect'], 'purple')
f.add_subplot(332)
histPlot(data_test['log_Aspect'], 'green')
#f.add_subplot(334)
#histPlot(data_test['Hillshade_3pm_boxcox'], 'gold')                    
f.add_subplot(333)
histPlot(data_test['sqr_Aspect'], 'c')

For aspect square root turned out to be the best transformation in terms of skeweness. 

Overall, the best transformations done here are square rt, and boxcox. log transformations did not proved to be benefitial

In [ ]:
data_test.drop(['log_Aspect'], axis=1,inplace=True)

In here we are transforming the ratios into a unit scale by dividing by its index. We do so because we think it is much easier to understand

In [ ]:
data_test['ratio_Hillshade_3pm'] = data_test['Hillshade_3pm']/255
data_test['ratio_Hillshade_Noon'] = data_test['Hillshade_Noon']/255
data_test['ratio_Hillshade_9am'] = data_test['Hillshade_9am']/255


<a id='5.2.8.7'></a>
### <font color=green> 5.2.8.7 Aspect in degrees <font>
### <font color=green> New Features  <font>
The azimuth is the angular direction of the sun, measured from north clockwise in degrees from 0 to 360. An Azimuth of 90 degrees is east.The Cut of values will be between for instance the middle of north and east.
    
We make a transformation so to get dummies for Aspect ordinal values: north, south, east and west

* Aspect_North: from 315 deg to 45 deg
* Aspect_East: from 45 deg to 135 deg
* Aspect_South: from 135 deg to 225 deg
* Aspect_West: from 225 deg to 315 deg    

<img src="angle_azimuth.png" width=400 height=200 align="center">
    
Source:https://www.pveducation.org/pvcdrom/properties-of-sunlight/azimuth-angle


<a id='5.2.8.8'></a>
### <font color=green> 5.2.8.8 Elevation <font>

No transformation is done as it is already very symetric distributed 


In [ ]:
print('\033[95m'+"Skew before transformation\n", data_test['Elevation'].skew(), 
      "\nmin\n", data_test['Elevation'].min(),
      "\nmax\n", data_test['Elevation'].max(),)

In [ ]:
data_test['binned_elevation'] = [math.floor(v/50.0) for v in data_test['Elevation']]

We are making more features byt summing and substracing different combinations similar in terms of units

Addition and Substraction on the same scale
Using for loop was giving us a bad performance hence we are using the features on the same scale which to add or substract 

In [ ]:
data_test['sqr_Road+Fire'] = (data_test['Horizontal_Distance_To_Roadways'] + data_test['Horizontal_Distance_To_Fire_Points'])**0.5
data_test['sqr_Road-Fire'] = (abs(data_test['Horizontal_Distance_To_Roadways'] - data_test['Horizontal_Distance_To_Fire_Points']))**0.5
data_test['sqr_Road+Hydro'] = (data_test['Horizontal_Distance_To_Roadways'] + data_test['Horizontal_Distance_To_Hydrology'])**0.5
data_test['sqr_Road-Hydro'] = (abs(data_test['Horizontal_Distance_To_Roadways'] - data_test['Horizontal_Distance_To_Hydrology']))**0.5
data_test['sqr_Hydro+Fire'] = (data_test['Horizontal_Distance_To_Hydrology'] + data_test['Horizontal_Distance_To_Fire_Points'])**0.5
data_test['sqr_Hydro-Fire'] = (abs(data_test['Horizontal_Distance_To_Hydrology'] - data_test['Horizontal_Distance_To_Fire_Points']))**0.5

data_test['sqr_Road+Fire+Hydro'] = (data_test['Horizontal_Distance_To_Roadways']  + data_test['Horizontal_Distance_To_Fire_Points'] + data_test['Horizontal_Distance_To_Hydrology'])**0.5

data_test['sqr_Ele+Road+Fire+Hydro'] = (data_test['Elevation'] + data_test['Horizontal_Distance_To_Roadways']  + data_test['Horizontal_Distance_To_Fire_Points'] + data_test['Horizontal_Distance_To_Hydrology'])**0.5

data_test['sqr_Ele+road'] = (data_test['Elevation'] + data_test['Horizontal_Distance_To_Roadways'])**0.5
data_test['sqr_Ele-road'] = (abs(data_test['Elevation'] - data_test['Horizontal_Distance_To_Roadways']))**0.5
data_test['sqr_Ele+fire'] = (data_test['Elevation'] + data_test['Horizontal_Distance_To_Fire_Points'])**0.5
data_test['sqr_Ele-fire'] = (abs(data_test['Elevation'] - data_test['Horizontal_Distance_To_Fire_Points']))**0.5
data_test['sqr_Ele+hydro'] = (data_test['Elevation'] + data_test['Horizontal_Distance_To_Hydrology'])**0.5
data_test['sqr_Ele-hydro'] = (abs(data_test['Elevation'] - data_test['Horizontal_Distance_To_Hydrology']))**0.5

### <font color=green> 5.2.9 Geoclimate grouping  <font>

#### <font color=green> 5.2.9.1 Climatic feature engineering to group soils  <font>

In the Kaggle competition, there is a reference to John A. Blackard which happened to be one geologist working for the forest federal US agency. In a co-authored paper, he gives further insights on the soil families with a list of codes. These are digits categorizing the soils according to climate and geology. We decide to take this valuable insight and engineer features around this dynamic so to cut down the number of soils

From original database donated by John A. Blackard

Code Designations:

Wilderness Areas:  	<br>

1 - Rawah Wilderness Area <br>
2 - Neota Wilderness Area  <br>
3 - Comanche Peak Wilderness Area<br>
4 - Cache la Poudre Wilderness Area<br>

Soil Types:             1 to 40 : based on the USFS Ecological
                        Landtype Units (ELUs) for this study area:<br>

  Study Code USFS ELU Code			Description<br>
	 1	   2702		Cathedral family - Rock outcrop complex, extremely stony.<br>
	 2	   2703		Vanet - Ratake families complex, very stony.<br>
	 3	   2704		Haploborolis - Rock outcrop complex, rubbly.<br>
	 4	   2705		Ratake family - Rock outcrop complex, rubbly.<br>
	 5	   2706		Vanet family - Rock outcrop complex complex, rubbly.<br>
	 6	   2717		Vanet - Wetmore families - Rock outcrop complex, stony.<br>
	 7	   3501		Gothic family.<br>
	 8	   3502		Supervisor - Limber families complex.<br>
	 9	   4201		Troutville family, very stony.<br>
	10	   4703		Bullwark - Catamount families - Rock outcrop complex, rubbly.<br>
	11	   4704		Bullwark - Catamount families - Rock land complex, rubbly.<br>
	12	   4744		Legault family - Rock land complex, stony.<br>
	13	   4758		Catamount family - Rock land - Bullwark family complex, rubbly.<br>
	14	   5101		Pachic Argiborolis - Aquolis complex.<br>
	15	   5151		unspecified in the USFS Soil and ELU Survey.<br>
	16	   6101		Cryaquolis - Cryoborolis complex.<br>
	17	   6102		Gateview family - Cryaquolis complex.<br>
	18	   6731		Rogert family, very stony.<br>
	19	   7101		Typic Cryaquolis - Borohemists complex.<br>
	20	   7102		Typic Cryaquepts - Typic Cryaquolls complex.<br>
	21	   7103		Typic Cryaquolls - Leighcan family, till substratum complex.<br>
	22	   7201		Leighcan family, till substratum, extremely bouldery.<br>
	23	   7202		Leighcan family, till substratum - Typic Cryaquolls complex.<br>
	24	   7700		Leighcan family, extremely stony.<br>
	25	   7701		Leighcan family, warm, extremely stony.<br>
	26	   7702		Granile - Catamount families complex, very stony.<br>
	27	   7709		Leighcan family, warm - Rock outcrop complex, extremely stony.<br>
	28	   7710		Leighcan family - Rock outcrop complex, extremely stony.<br>
	29	   7745		Como - Legault families complex, extremely stony.<br>
	30	   7746		Como family - Rock land - Legault family complex, extremely stony.<br>
	31	   7755		Leighcan - Catamount families complex, extremely stony.<br>
	32	   7756		Catamount family - Rock outcrop - Leighcan family complex, extremely stony.<br>
	33	   7757		Leighcan - Catamount families - Rock outcrop complex, extremely stony.<br>
	34	   7790		Cryorthents - Rock land complex, extremely stony.<br>
	35	   8703		Cryumbrepts - Rock outcrop - Cryaquepts complex.<br>
	36	   8707		Bross family - Rock land - Cryumbrepts complex, extremely stony.<br>
	37	   8708		Rock outcrop - Cryumbrepts - Cryorthents complex, extremely stony.<br>
	38	   8771		Leighcan - Moran families - Cryaquolls complex, extremely stony.<br>
	39	   8772		Moran family - Cryorthents - Leighcan family complex, extremely <br>stony.
	40	   8776		Moran family - Cryorthents - Rock land complex, extremely stony.<br>

        Note:   First digit:  climatic zone       Second digit:  geologic zones
                1.  lower montane dry             1.  alluvium
                2.  lower montane                 2.  glacial
                3.  montane dry                   3.  shale
                4.  montane                       4.  sandstone
                5.  montane dry and montane       5.  mixed sedimentary
                6.  montane and subalpine         6.unspecified in the USFS ELU Survey
                7.  subalpine                     7.  igneous and metamorphic
                8.  alpine                        8.  volcanic

The USFD, an American federal agency for forest service dependent on the department of agriculture has classified soil types according to __climatic zone (first digit)__ and __geology (second digit)__. Because of this, we believe a similar classification can be artificially engineered grouping all similar soils in 7 categories for climate (there is no lower montane dry soils) and 4 for geology (we do not take into consideration shale, sandstone, volcanic or unspecified)

#### <font color=green> 5.2.9.2 Climatic Zone feature engineering to group soils  <font>

In [ ]:
data_test["Lower_Montane_Climate"] = data_test.loc[:,data_test.columns.str.contains("^Soil_Type[23456]$")].max(axis=1)

In [ ]:
data_test['Montane_Dry_Climate'] =data_test.loc[:,data_test.columns.str.contains("^Soil_Type[78]$")].max(axis=1)

In [ ]:
data_test['Montane_Climate'] =data_test.loc[:,data_test.columns.str.contains("^Soil_Type[1][0123]$|Soil_Type[9]$")].max(axis=1)

In [ ]:
data_test['Montane_Dry_and_Montane_Climate'] =data_test.loc[:,data_test.columns.str.contains("^Soil_Type[1][45]$")].max(axis=1)

In [ ]:
data_test['Montante_and_Subalpine_Climate'] =data_test.loc[:,data_test.columns.str.contains("^Soil_Type[1][678]$")].max(axis=1)


In [ ]:
data_test['Subalpine_Climate'] =data_test.loc[:,data_test.columns.str.contains("^Soil_Type19$|^Soil_Type[2][0-9]$|^Soil_Type[3][0-4]$")].max(axis=1)


In [ ]:
data_test['Alpine_Climate'] =data_test.loc[:,data_test.columns.str.contains("^Soil_Type[3][56789]$|Soil_Type40")].max(axis=1)

#### <font color=green> 5.2.9.2 Geological feature engineering to group soils  <font>

        Note:   First digit:  climatic zone             Second digit:  geologic zones
                1.  lower montane dry                   1.  alluvium
                2.  lower montane                       2.  glacial
                3.  montane dry                         3.  shale
                4.  montane                             4.  sandstone
                5.  montane dry and montane             5.  mixed sedimentary
                6.  montane and subalpine               6.  unspecified in the USFS ELU Survey
                7.  subalpine                           7.  igneous and metamorphic
                8.  alpine                              8.  volcanic

In [ ]:
data_test['Alluvium_Soil'] = data_test.loc[:,data_test.columns.str.contains("^Soil_Type[1][45679]$|^Soil_Type[2][01]$")].max(axis=1)

In [ ]:
data_test['Glacial_Soil'] =data_test.loc[:,data_test.columns.str.contains("^Soil_Type[9]$|^Soil_Type[2][23]$")].max(axis=1)

In [ ]:
data_test['Mixed_Sedimentary_Soil'] =data_test.loc[:,data_test.columns.str.contains("^Soil_Type[7-8]$")].max(axis=1)

In [ ]:
data_test['Igneus_and_Metamorphic_Soil'] =data_test.loc[:,data_test.columns.str.contains("^Soil_Type[1-6]$|^Soil_Type[1][01238]$|^Soil_Type[3-4]\d$|^Soil_Type[2][4-9]$")].max(axis=1)

In [ ]:
data_test.head()

Based on the medium paper "Preprocessing: Why you should generate polynominal features first before standardizing" mention it is not good practice to standardize the variablesbefore before PolynominalFeatures. This should be done after to not loss the signal of the variables.  

### Advanced Factorization

The numerical values present a level of detail that may be much more fine-grained than we need. For instance, the soil level can be represented by different categories (soil family, complex or stony/rubberly). We aggregate the data up which can help to avoid overfitting when the data is more aggregate.

__We played with these features but reached a final conclusion of not to considering any more families and complex type. But Stonyness,climate and geogology. For purposes of the assignment we show the code as a RAWNBCONVERT__

The only family grouping we do on soils are the ones above, on geological and climate grounds

### <font color=green> 5.6 Soil Type Family  <font>

Using Discretization to bin the soil variable to the family type.<br>

__Cathedral__ <br>
1 Cathedral family - Rock outcrop complex, extremely stony.<br>

__Ratake__ <br>
2 Vanet - Ratake families complex, very stony.<br>
4 Ratake family - Rock outcrop complex, rubbly.<br>

__Vanet__<br>
5 Vanet family - Rock outcrop complex complex, rubbly.<br>

__Wetmore__<br>
6 Vanet - Wetmore families - Rock outcrop complex, stony.<br>

__Gothic__<br>
7 Gothic family.<br>
                    
__Limber__ <br>
8 Supervisor - Limber families complex. <br>

__Troutville__<br>
9 Troutville family, very stony.<br>

__Legault__<br>
12 Legault family - Rock land complex, stony.<br>
29 Como - Legault families complex, extremely stony.<br>

__Gateview__ <br>
17 Gateview family - Cryaquolis complex.<br>

__Rogert__<br>
18 Rogert family, very stony.<br>


__Como__<br>
30 Como family - Rock land - Legault family complex, extremely stony.<br>

__Bross__<br>
36 Bross family - Rock land - Cryumbrepts complex, extremely stony.<br>



__Catamount__<br>
10 Bullwark - Catamount families - Rock outcrop complex, rubbly.<br>
11 Bullwark - Catamount families - Rock land complex, rubbly.<br>
13 Catamount family - Rock land - Bullwark family complex, rubbly.<br>
26 Granile - Catamount families complex, very stony.<br>
32 Catamount family - Rock outcrop - Leighcan family complex, extremely stony.<br>
31 Leighcan - Catamount families complex, extremely stony.<br>
33 Leighcan - Catamount families - Rock outcrop complex, extremely stony.<br>

__Leighcan__<br>
21 Typic Cryaquolls - Leighcan family, till substratum complex.<br>
22 Leighcan family, till substratum, extremely bouldery.<br>
23 Leighcan family, till substratum - Typic Cryaquolls complex.<br>
24 Leighcan family, extremely stony.<br>
25 Leighcan family, warm, extremely stony.<br>
27 Leighcan family, warm - Rock outcrop complex, extremely stony.<br>
28 Leighcan family - Rock outcrop complex, extremely stony.<br>

__Moran__<br>
38 Leighcan - Moran families - Cryaquolls complex, extremely stony.<br>
39 Moran family - Cryorthents - Leighcan family complex, extremely stony.<br>
40 Moran family - Cryorthents - Rock land complex, extremely stony.<br>

__Others__<br> 
3 Haploborolis - Rock outcrop complex, rubbly.<br>
15 unspecified in the USFS Soil and ELU Survey.<br>
37 Rock outcrop - Cryumbrepts - Cryorthents complex, extremely stony.<br>
34 Cryorthents - Rock land complex, extremely stony.<br>
35 Cryumbrepts - Rock outcrop - Cryaquepts complex.<br>
20 Typic Cryaquepts - Typic Cryaquolls complex.<br>
14 Pachic Argiborolis - Aquolis complex.<br>
16 Cryaquolis - Cryoborolis complex.<br>
19 Typic Cryaquolis - Borohemists complex.<br>

__NOT USED__

We will group the soil types according to their family and according to the complex and stonyness

__NOT USED__

__This has been included__

Note: Soil type is a single variable which has been one-hot encoded presumably , so we will reverse engineer the soil type. We will eventually drop the original soil type columns which has the added effect of significantly reducing the total number of features.

In [ ]:
# Original soil features
soil_features = [f'Soil_Type{i}' for i in range(1,41)]

In [ ]:
# Drop original soil features
data_test.drop(columns = soil_features, inplace = True)

In [ ]:
#test if elevation makes a difference to take out with the new interaction model improves
data_test = data_test.drop(['Elevation^2'], axis = 1)
data_test = data_test.drop(['Elevation'], axis = 1)

In [ ]:
data_test

Removing the original scaled variables did not improve nor worsen the model. Since it does not change much the score, we remove it as we have it double in the model with the scaled features. 

### Locking all features in a csv

For the shake of efficiency, we create a csv file to reuse also later on in part III

In [ ]:
data_test_conformed = data_test[['Id','Wilderness_Area4',
 'Subalpine_Climate',
 'Alpine_Climate',
 'sqr_Road-Hydro',
 'sqr_Road-Fire',
 'sqr_Road+Fire',
 'binned_elevation',
 'Wilderness_Area3',
 'Wilderness_Area1',
 'Montane_Climate',
 'Lower_Montane_Climate',
 'Horizontal_Distance_To_Roadways_x_Horizontal_Distance_To_Fire_Points',
 'sqr_Road+Fire+Hydro',
 'sqr_Ele-hydro',
 'sqr_Ele+road',
 'sqr_Ele+hydro',
 'sqr_Ele+fire',
 'sqr_Ele+Road+Fire+Hydro',
 'Horizontal_Distance_To_Roadways^2',
 'Hillshade_9am_x_Hillshade_Noon',
 'Elevation_x_Hillshade_Noon',
 'Elevation_x_Hillshade_9am',
 'sqr_Road+Hydro',
 'sqr_Ele-road',
 'sqr_Ele-fire',
 'sqr_Distance_To_Hydrology',
 'Sqr_Horizontal_Distance_To_Roadways',
 'Igneus_and_Metamorphic_Soil',
 'Horizontal_Distance_To_Roadways_x_Hillshade_Noon',
 'Horizontal_Distance_To_Roadways_x_Hillshade_9am',
 'Horizontal_Distance_To_Hydrology_x_Hillshade_3pm',
 'Hillshade_Noon_x_Horizontal_Distance_To_Fire_Points',
 'Hillshade_9am_boxcox',
 'Elevation_x_Horizontal_Distance_To_Roadways',
 'Elevation_x_Horizontal_Distance_To_Hydrology']]

This code below have all features that we did in the feature engineering part and transport this to csv before we actually use the feature selection methods and select only few variables. 

In [ ]:
#Only X_Train replacement
data_test_conformed.to_csv('selected_features_test.csv')
